#Instalação Bibliotecas

In [ ]:
pip install pyspark

In [ ]:
pip install fsspec

In [ ]:
pip install gcsfs

#Bibliotecas

In [ ]:
#Importando funcções padrão. Função Window também importada porque será usado mais na frente
import os
from google.cloud import storage

from pyspark.sql import SparkSession, Window
spark = SparkSession.builder.appName('GCSFilesRead').getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.functions import to_timestamp, to_date

from pyspark.sql.types import *
import pandas as pd

In [ ]:
dfspark = pd.read_csv('gs://projetofinalgrupo8/saida/linkedin_tratado_pyspark.csv', sep=',',encoding='UTF-8')
df = spark.createDataFrame(dfspark).createOrReplaceTempView("leitura_linkedin")

#Consultas

**00.Consulta geral tabela**

In [ ]:
spark.sql("SELECT * FROM leitura_linkedin LIMIT 10").show()

+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+--------+---------+------------+-------+--------------+----------+----------------+----------------------+
|tempo_cargo_anterior|id_usuario|promocoes|dias_cargo_anterior|idade|desfoque|raiva|desgosto|medo|felicidade|neutro|triste|surpresa|   etnia|   genero|      oculos|sorriso| nacionalidade|seguidores|qualidade_imagem|tempo_cargo_anterior\t|
+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+--------+---------+------------+-------+--------------+----------+----------------+----------------------+
|                 2.0|      3008|        1|                457|   37|    0.42| 76.6|     0.7| 0.7|     99.04|  0.09|  0.01|    0.09|Asiático|Masculino|Oculos_comum|  77.54|asiatico_leste|       420|           67.66|                   2.0|
|                 1.0|      3010|        1| 

**01.Pessoas com mais de 50 anos que usam óculos escuros**

---



In [ ]:
spark.sql("SELECT * FROM leitura_linkedin WHERE oculos = 'Oculos_escuros' AND idade > 50 ORDER BY idade DESC LIMIT 10").show()

+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+------+---------+--------------+-------+-------------+----------+----------------+----------------------+
|tempo_cargo_anterior|id_usuario|promocoes|dias_cargo_anterior|idade|desfoque|raiva|desgosto|medo|felicidade|neutro|triste|surpresa| etnia|   genero|        oculos|sorriso|nacionalidade|seguidores|qualidade_imagem|tempo_cargo_anterior\t|
+--------------------+----------+---------+-------------------+-----+--------+-----+--------+----+----------+------+------+--------+------+---------+--------------+-------+-------------+----------+----------------+----------------------+
|                 1.0|      2546|        1|               1795|   74|    0.22| 12.7|     1.8| 2.8|     96.59|  0.07|  3.15|    0.02|Branco|Masculino|Oculos_escuros|  62.73|       ingles|       522|           94.29|                   1.0|
|                 1.0|     12557|        1|     

**02.Consulta pessoas negras entre 16 e 20 anos com suas respectivas nacionalidades e número de seguidores**

In [ ]:
spark.sql("SELECT idade, genero, nacionalidade, seguidores, qualidade_imagem \
           FROM leitura_linkedin \
           WHERE (idade >= 16 AND idade <= 20 AND etnia = 'Negro') \
           ORDER BY seguidores DESC").show()

+-----+---------+--------------+----------+----------------+
|idade|   genero| nacionalidade|seguidores|qualidade_imagem|
+-----+---------+--------------+----------+----------------+
|   18| Feminino|  asiatico_sul|      1952|           93.31|
|   19|Masculino|asiatico_leste|      1740|           93.37|
|   18|Masculino|  asiatico_sul|      1594|           93.12|
|   20| Feminino|      africano|      1460|           82.49|
|   18|Masculino|  asiatico_sul|      1293|           88.53|
|   20|Masculino|  asiatico_sul|       916|           77.78|
|   16|Masculino|  asiatico_sul|       867|            87.7|
|   20|Masculino|        ingles|       830|           92.35|
|   20|Masculino|       europeu|       772|           87.31|
|   19|Masculino|  asiatico_sul|       743|            0.46|
|   20|Masculino|     muculmano|       679|           90.02|
|   19|Masculino|  asiatico_sul|       652|           51.78|
|   20|Masculino|  asiatico_sul|       600|            6.56|
|   17|Masculino|asiatic

**03.Consulta pessoas que usam ou não óculos por nacionalidade**

In [ ]:
spark.sql("SELECT nacionalidade, COUNT(nacionalidade) AS qty_nacionalidade, SUM(case when oculos = 'Oculos_comum' then 1 else 0 end) AS usa_oculos, \
                  SUM(case when oculos = 'Oculos_escuros' then 1 else 0 end) AS usa_oculos_escuros, \
                  SUM(case when oculos = 'Nenhum' then 1 else 0 end) AS sem_oculos \
           FROM leitura_linkedin \
           GROUP BY nacionalidade \
           ORDER BY qty_nacionalidade DESC").show()

+--------------+-----------------+----------+------------------+----------+
| nacionalidade|qty_nacionalidade|usa_oculos|usa_oculos_escuros|sem_oculos|
+--------------+-----------------+----------+------------------+----------+
|        ingles|             4491|       755|                78|      3658|
|  asiatico_sul|             1947|       487|                88|      1372|
|       europeu|             1893|       324|                39|      1530|
|asiatico_leste|             1220|       432|                17|       771|
|     hispanico|             1177|       186|                27|       964|
|     muculmano|              675|       149|                23|       503|
|       nordico|              246|        59|                 4|       183|
|      africano|               85|        11|                 1|        73|
|         grego|               62|         8|                 0|        54|
|         judeu|               45|         6|                 2|        37|
+-----------

**04.Consulta média de seguidores e quantidade de mulheres influenciadoras no 
LinkedIn por nacionalidade**

In [ ]:
spark.sql("SELECT nacionalidade, COUNT(nacionalidade) AS qty_mulheres, ROUND(MEAN(seguidores), 2) AS media_seguidores \
           FROM leitura_linkedin \
           WHERE genero = 'Feminino' \
           GROUP BY nacionalidade \
           ORDER BY media_seguidores DESC").show()

+--------------+------------+----------------+
| nacionalidade|qty_mulheres|media_seguidores|
+--------------+------------+----------------+
|     muculmano|         106|         1025.08|
|       europeu|         470|         1017.66|
|      africano|          15|           999.0|
|  asiatico_sul|         328|          980.11|
|         grego|          16|          943.31|
|        ingles|        1237|          939.12|
|     hispanico|         367|           821.1|
|         judeu|           6|           786.5|
|asiatico_leste|         392|           698.7|
|       nordico|          56|          695.02|
+--------------+------------+----------------+



**05.Consulta quantidade de homens e mulheres por etnia**

In [ ]:
spark.sql("SELECT DISTINCT etnia, COUNT(etnia) AS qty_etnia, SUM(case when genero = 'Masculino' then 1 else 0 end) AS qty_homens, \
                                  SUM(case when genero = 'Feminino' then 1 else 0 end) AS qty_mulheres  \
           FROM leitura_linkedin \
           GROUP BY etnia").show()

+--------+---------+----------+------------+
|   etnia|qty_etnia|qty_homens|qty_mulheres|
+--------+---------+----------+------------+
|   Negro|      942|       821|         121|
|Asiático|     1963|      1346|         617|
|  Branco|     8936|      6681|        2255|
+--------+---------+----------+------------+



**06.Consulta número de homens e mulheres por etnia, nacionalidade e sexo**




In [ ]:
spark.sql("SELECT DISTINCT etnia, nacionalidade, SUM(case when genero = 'Masculino' then 1 else 0 end) AS qty_homens, \
                                  SUM(case when genero = 'Feminino' then 1 else 0 end) AS qty_mulheres  \
           FROM leitura_linkedin \
           GROUP BY etnia, nacionalidade \
           ORDER BY nacionalidade, etnia").show(50)

+--------+--------------+----------+------------+
|   etnia| nacionalidade|qty_homens|qty_mulheres|
+--------+--------------+----------+------------+
|Asiático|      africano|         6|           4|
|  Branco|      africano|        39|           6|
|   Negro|      africano|        25|           5|
|Asiático|asiatico_leste|       687|         321|
|  Branco|asiatico_leste|       111|          62|
|   Negro|asiatico_leste|        30|           9|
|Asiático|  asiatico_sul|       129|          27|
|  Branco|  asiatico_sul|      1061|         259|
|   Negro|  asiatico_sul|       429|          42|
|Asiático|       europeu|        93|          45|
|  Branco|       europeu|      1279|         413|
|   Negro|       europeu|        51|          12|
|Asiático|         grego|         0|           2|
|  Branco|         grego|        45|          14|
|   Negro|         grego|         1|           0|
|Asiático|     hispanico|       126|          72|
|  Branco|     hispanico|       644|         284|


**07.Consulta quantidade de homens e mulheres entre 36 e 40 anos de acordo com a etnia**


In [ ]:
spark.sql("SELECT DISTINCT idade, etnia, SUM(case when genero = 'Masculino' then 1 else 0 end) AS qty_homens, \
                                  SUM(case when genero = 'Feminino' then 1 else 0 end) AS qty_mulheres  \
           FROM leitura_linkedin \
           WHERE idade >= 36 AND idade <= 40 \
           GROUP BY idade, etnia \
           ORDER BY etnia, idade").show(50)

+-----+--------+----------+------------+
|idade|   etnia|qty_homens|qty_mulheres|
+-----+--------+----------+------------+
|   36|Asiático|        59|          15|
|   37|Asiático|        45|          30|
|   38|Asiático|        50|          11|
|   39|Asiático|        44|          17|
|   40|Asiático|        64|          17|
|   36|  Branco|       166|          51|
|   37|  Branco|       188|          63|
|   38|  Branco|       185|          62|
|   39|  Branco|       200|          64|
|   40|  Branco|       247|          68|
|   36|   Negro|        36|           2|
|   37|   Negro|        32|           9|
|   38|   Negro|        31|           6|
|   39|   Negro|        21|           4|
|   40|   Negro|        42|           0|
+-----+--------+----------+------------+



**08.Consulta idades (mais velho, média e mais novo) por etnia e quantos usam ou não usam óculos**



In [ ]:
spark.sql("SELECT DISTINCT etnia, MAX(idade) AS mais_velho, ROUND(MEAN(idade), 2) AS media_idade, MIN(idade) AS mais_novo, \
                           SUM(case when oculos = 'Oculos_comum' then 1 else 0 end) AS usa_oculos, \
                           SUM(case when oculos = 'Oculos_escuros' then 1 else 0 end) AS usa_oculos_escuros, \
                           SUM(case when oculos = 'Nenhum' then 1 else 0 end) AS sem_oculos \
           FROM leitura_linkedin \
           GROUP BY etnia \
           ORDER BY etnia").show(50)

+--------+----------+-----------+---------+----------+------------------+----------+
|   etnia|mais_velho|media_idade|mais_novo|usa_oculos|usa_oculos_escuros|sem_oculos|
+--------+----------+-----------+---------+----------+------------------+----------+
|Asiático|        75|      36.37|       16|       624|                68|      1271|
|  Branco|        80|      44.84|       16|      1556|               179|      7201|
|   Negro|        76|      40.09|       16|       237|                32|       673|
+--------+----------+-----------+---------+----------+------------------+----------+

